# Advanced Querying Mongo

In [1]:
%pip install pymongo   

     -------------------------------------- 382.5/382.5 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 283.7/283.7 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient()

In [4]:
str_conn='mongodb://localhost:27017'

cursor=MongoClient(str_conn)

cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
cursor.list_database_names()

['admin', 'companies', 'config', 'local']

In [6]:
db=client.companies

In [12]:
colec=db.IRONHACK

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [14]:
#list(colec.find())

In [21]:
query={'name': "Babelgum"} #select all que contenga babalgum en su nombre
select={"_id":0, "name":1} #entre corchetes always recuerda son diccionarios y este select como en SQL, muestrame de la condición, la columna name.
#_id FALSE/0 porque por default te lo va a mostrar
list(colec.find(query,select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [22]:
field_names = colec.find_one().keys()
print(list(field_names))     #esta formu es para devolvernos basicamente te devuelve las keys en una variable field names

['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners']


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [27]:
query={'$and': [{'founded_year': {'$lt': 2005}},
                {'founded_year': {'$gt': 2000}}]}

select={'_id':0, 'name':1, 'founded_year':1 }
results = list(colec.find(query, select))
#results

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [43]:
query={'$and': [{'ipo.valuation_amount': {'$gt': 10e7}},
                {'founded_year': {'$lt': 2010}}]}

select={'_id':0, 'name':1, 'ipo':1 }
results = list(colec.find(query, select))
#results

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [41]:
query={'$and': [{'number_of_employees': {'$lt': 1000}},
                {'founded_year': {'$lt': 2005}}]}

select={'_id':0, 'name':1, 'number_of_employees':1, 'founded_year':1 }
results = list(colec.find(query, select).sort('number_of_employees', -1).limit(20))
#results

### 6. All the companies that don't include the `partners` field.

In [34]:
query={'partners':{'$exists': False}}
select={'_id':0, 'name':1,}
results = list(colec.find(query, select))
results

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [37]:
query={'category_code': None}
select={'_id':0, 'name':1,}
results = list(colec.find(query, select))
#results

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [39]:
query={'$and': [{'number_of_employees': {'$lt': 1000}},
                {'number_of_employees': {'$lt': 100}}]}

select={'_id':0, 'name':1, 'number_of_employees':1}
results = list(colec.find(query, select).sort('number_of_employees', -1).limit(20))
#results

### 9. Order all the companies by their IPO price in a descending order.

In [50]:
select={'_id': 0, 'name':1, 'ipo':1}
results=list(colec.find({},select).sort('ipo.valuation_amount', -1))
#results

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [56]:
select={'_id': 0, 'name':1, 'number_of_employees':1}
results=list(colec.find({},select).sort('number_of_employees', -1).limit(10))
#results

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [60]:
query={'founded_month': {'$gte':7}}
select={'_id': 0, 'name':1, 'founded_month':1}
results=list(colec.find(query,select).limit(1000))
#results

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [64]:
query={'$and': [{'founded_year': {'$lt': 2000}},
                {'acquisition.price_amount': {'$gt': 10000000
                }}]}
select={'_id': 0, 'name':1, 'founded_year':1,'acquisition.price_amount':1}
results=list(colec.find(query,select).sort('acquisition.price_amount',1))
#results

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [68]:
query = {'acquisition.acquired_year': {'$gt': 2010}}
select = {'_id': 0, 'name': 1, 'acquisition': 1}
results = list(colec.find(query, select).sort('acquisition.price_amount', -1))
#results


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [71]:
select = {'_id': 0, 'name': 1, 'founded_year': 1}
results = list(colec.find({}, select).sort('founded_year', -1))
#results

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [73]:
query={'founded_day': {'$lte': 7}}
select={'_id': 0, 'name':1, 'acquisition.price_amount':1}
results=list(colec.find(query,select).sort('acquisition.price_amount',-1).limit(10))
#results

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [74]:
query={'$and': [{'number_of_employees': {'$gt': 4000}},
                {'category_code': 'web'}]}
select={'_id': 0, 'name':1, 'number_of_employees':1}
results=list(colec.find(query,select).sort('number_of_employees',1))
results

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [75]:
query={'$and': [{'acquisition.price_amount': {'$gt': 10000000}},
                {'acquisition.price_currency_code': 'EUR'}]}
select={'_id': 0, 'name':1}
results=list(colec.find(query,select))
results

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [79]:
query={'acquisition.acquired_month': {'$lte':3}}
select={'_id': 0, 'name':1, 'acquisition':1}
results=list(colec.find(query,select).limit(10))
#results

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code